In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.schema import HumanMessage
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda
from operator import itemgetter

import json
from langchain.docstore.document import Document


# Data Preparation

## Import scraped data from scraper

In [2]:
# from scraper import json_list

In [3]:
# json_list[:5]

In [4]:
# # เซฟทั้งหมดเป็นไฟล์เดียว
# with open("data.json", "w", encoding="utf-8") as f:
#     json.dump(json_list, f, ensure_ascii=False, indent=2)


## อ่านข้อมูลจาก json มาเก็บในลิสต์

In [5]:
json_list_x = []
with open("data.json", "r", encoding="utf-8") as f:
    json_list_x = json.load(f)


## Change json_list to Document

In [6]:
documents = []
for item in json_list_x:
    content = f"subject: {item['subject']}\n gender: {item['gender']}\n age: {item['age']}\n symptoms: {', '.join(item['symptoms'])}\n question: {item['question']}\n answer: {item['answer']}"
    documents.append(Document(page_content=content, metadata={"source": "scraper"}))



## Embeding and Store to VectorDB

In [7]:
embedder = OllamaEmbeddings(model="bge-m3")


C:\Users\Thirachai\AppData\Local\Temp\ipykernel_7856\2229192303.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedder = OllamaEmbeddings(model="bge-m3")


In [8]:
#vector_store = FAISS.from_documents(documents, embedder)

vector_store = FAISS.load_local("D:/faiss_storage", 
                                embedder,
                                allow_dangerous_deserialization=True
)
print("Loaded vector store from disk")

Loaded vector store from disk


In [9]:
retriever = vector_store.as_retriever()



# Create chat

In [10]:
llm = ChatOllama(model="scb10x/llama3.1-typhoon2-8b-instruct",
                    temperature=0.4)
# from medpalm.model import MedPalm
# llm = MedPalm()
print(llm)

model='scb10x/llama3.1-typhoon2-8b-instruct' temperature=0.4


C:\Users\Thirachai\AppData\Local\Temp\ipykernel_7856\2354700130.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="scb10x/llama3.1-typhoon2-8b-instruct",


In [11]:


from huggingface_hub import hf_hub_download
from llama_cpp import Llama

model_name = "aaditya/OpenBioLLM-Llama3-8B-GGUF"
model_file = "openbiollm-llama3-8b.Q5_K_M.gguf"

model_path = hf_hub_download(model_name,
                             filename=model_file,
                             local_dir='/content')
print("My model path: ", model_path)
llm_x = Llama(model_path=model_path,
            n_gpu_layers=-1,
            temperature=0.5,
            )

C:\Users\Thirachai\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from \content\openbiollm-llama3-8b.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 

My model path:  \content\openbiollm-llama3-8b.Q5_K_M.gguf


Exception ignored on calling ctypes callback function <function llama_log_callback at 0x00000170FF6FD1C0>:
Traceback (most recent call last):
  File "C:\Users\Thirachai\AppData\Roaming\Python\Python313\site-packages\llama_cpp\_logger.py", line 29, in llama_log_callback
    @llama_cpp.llama_log_callback
KeyboardInterrupt: 
llama_model_loader: - kv  15:                      tokenizer.ggml.scores arr[f32,128256]  = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  16:                  tokenizer.ggml.token_type arr[i32,128256]  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  17:                      tokenizer.ggml.merges arr[str,280147]  = ["Ġ Ġ", "Ġ ĠĠĠ", "ĠĠ ĠĠ", "...
llama_model_loader: - kv  18:                tokenizer.ggml.bos_token_id u32              = 128000
llama_model_loader: - kv  19:                tokenizer.ggml.eos_token_id u32              = 128001
llama_model_loader: - kv  20:            tokenizer.ggml.padding_token_id u32             

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "คุณคือผู้แพทย์เชี่ยวชาญด้านสุขภาพที่มีความรู้ลึกซึ้งทั้งจากประสบการณ์และฐานข้อมูลทางการแพทย์ "
     "คุณจะต้องให้คำแนะนำที่แม่นยำและปลอดภัย โดยเริ่มต้นด้วยประโยค 'คำตอบจากแพทย์ผู้เชี่ยวชาญ' "
     "จากนั้นให้คำตอบแบ่งเป็นหัวข้อ: สาเหตุที่เป็นไปได้, แนวทางการดูแลเบื้องต้น, คำแนะนำเพิ่มเติม "
     " ใส่ข้อความ 'คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ \nhttps://bit.ly/doctor-refer-agnos' ในช่วงท้ายของการสนทนา"),
    
    ("human", 
     "มีผู้ป่วยที่มีข้อมูลดังนี้:\n"
     "- หัวข้อ: {subject}\n"
     "- เพศ: {gender}\n"
     "- อายุ: {age} ปี\n"
     "- อาการ: {symptoms}\n"
     "- คำถาม: {question}\n"
     "ข้อมูลจากฐานข้อมูลที่เกี่ยวข้อง:\n{context}\n"
     "กรุณาวิเคราะห์และให้คำตอบอย่างละเอียด โดยอ้างอิงจากทั้งความรู้ของคุณและข้อมูลที่ให้มาในส่วนของ context อย่างรอบคอบ"
     "หากข้อมูลใน context ไม่เพียงพอ ให้ใช้ความรู้ทั่วไปของคุณในการตอบคำถามอย่างระมัดระวังและแนะนำให้ผู้ป่วยไปพบแพทย์ผู้เชี่ยวชาญ"
     )
])

In [ ]:
from langchain.schema.runnable import RunnableLambda

def fill_missing_fields(inputs):
    return {
        "subject": inputs.get("subject", "ไม่ระบุ"),
        "gender": inputs.get("gender", "ไม่ระบุ"),
        "age": inputs.get("age", "ไม่ระบุ"),
        "symptoms": inputs.get("symptoms", "ไม่ระบุ"),
        "question": inputs.get("question", "ไม่ระบุ"),
        "context": inputs.get("context", "ไม่มีข้อมูลจากฐานข้อมูล")
    }

input_preprocessor = RunnableLambda(fill_missing_fields)


In [ ]:

def chat_to_string(chat_messages):
    return chat_messages.to_string()


In [ ]:
rag_chain = (
    {
        "context": retriever,
        "subject": RunnablePassthrough(),
        "gender": RunnablePassthrough(),
        "age": RunnablePassthrough(),
        "symptoms": RunnablePassthrough(),
        "question": RunnablePassthrough(),

    }
    | input_preprocessor
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain_x = (
    {
        "context": retriever,
        "subject": RunnablePassthrough(),
        "gender": RunnablePassthrough(),
        "age": RunnablePassthrough(),
        "symptoms": RunnablePassthrough(),
        "question": RunnablePassthrough(),

    }
    | input_preprocessor
    | prompt
    | RunnableLambda(chat_to_string)
    | llm_x
    | RunnableLambda(lambda x: x["text"])
    | StrOutputParser()
)

In [ ]:
data_input =[ {
    "subject": "อาการปวดหัวเรื้อรัง",
    "gender": "หญิง",
    "age": "35",
    "symptoms": "ปวดหัวบริเวณขมับทั้งสองข้างมาเกือบทุกวัน โดยเฉพาะช่วงบ่าย รู้สึกตึงๆ และมีอาการคลื่นไส้ร่วมด้วย",
    "question": "อาการแบบนี้เกิดจากอะไร และควรดูแลตัวเองอย่างไร?"
},
{
    "subject": "การนอนหลับในผู้สูงอายุ",
    "gender": "ชาย",
    "age": "68",
    "symptoms": "นอนหลับไม่สนิท ตื่นบ่อยตอนกลางคืน และรู้สึกไม่สดชื่นตอนเช้า",
    "question": "ควรปรับพฤติกรรมหรือมีวิธีดูแลสุขภาพอย่างไรให้หลับดีขึ้น?"
},
{
    "subject": "ภาวะขาดวิตามินบี 12 (Vitamin B12 deficiency)",
    "gender": "หญิง",
    "age": "29",
    "symptoms": [
      "เหนื่อยง่าย",
      "มือเท้าชา",
      "เวียนหัว",
      "ลืมง่าย",
      "อ่อนแรง"
    ],
    "question": "ช่วงนี้รู้สึกเหนื่อยง่ายมากค่ะ เดินนิดเดียวก็หอบ มือเท้าชาบ่อย ๆ โดยเฉพาะตอนตื่นนอน รู้สึกเวียนหัวและลืมง่ายขึ้นเรื่อย ๆ อยากทราบว่าอาการแบบนี้เกิดจากอะไร และควรตรวจอะไรบ้างคะ?"
  },
  {
    "subject": "โรคกรดไหลย้อน (GERD)",
    "gender": "ชาย",
    "age": "34",
    "symptoms": [
      "แสบร้อนกลางอก",
      "เรอเปรี้ยว",
      "ไอเรื้อรัง",
      "แน่นหน้าอก",
      "กลืนลำบาก"
    ],
    "question": "ผมมีอาการแสบร้อนกลางอกหลังทานอาหาร โดยเฉพาะอาหารเผ็ดหรือมัน ๆ เรอเปรี้ยวบ่อยมาก และบางครั้งรู้สึกแน่นหน้าอกจนหายใจไม่สะดวก อยากทราบว่าเป็นกรดไหลย้อนหรือเปล่า และควรปรับพฤติกรรมยังไงครับ?"
  },
  {
    "subject": "ภาวะถุงน้ำรังไข่หลายใบ (PCOS)",
    "gender": "หญิง",
    "age": "26",
    "symptoms": [
      "ประจำเดือนมาไม่ปกติ",
      "สิวขึ้นเยอะ",
      "ขนดก",
      "น้ำหนักขึ้นง่าย",
      "ปวดท้องน้อย"
    ],
    "question": "ประจำเดือนมาไม่ตรงเวลาเลยค่ะ บางเดือนก็ไม่มาเลย สิวขึ้นเยอะมากทั้งที่ดูแลผิวดีแล้ว ขนตามตัวก็ดูเยอะขึ้น น้ำหนักขึ้นง่ายมากทั้งที่กินเท่าเดิม อยากทราบว่าอาการแบบนี้เกี่ยวกับฮอร์โมนหรือเปล่า และควรไปตรวจอะไรบ้างคะ?"
  },
  {
    "subject": "โรคภูมิแพ้อากาศ (Allergic Rhinitis)",
    "gender": "ชาย",
    "age": "18",
    "symptoms": [
      "จามบ่อย",
      "คัดจมูก",
      "น้ำมูกไหล",
      "คันจมูก",
      "ตาแดง"
    ],
    "question": "ผมมีอาการจามบ่อยมาก โดยเฉพาะตอนเช้า ๆ หรือเวลาอยู่ในห้องแอร์ น้ำมูกไหล คัดจมูก และบางครั้งตาแดงคันร่วมด้วย อยากทราบว่าเป็นภูมิแพ้อากาศหรือเปล่า และมีวิธีดูแลตัวเองยังไงบ้างครับ?"
  }
]

In [ ]:
my_question = rag_chain.invoke({    
    "subject": "กรดไหลย้อน อาหารไม่ย่อย",
    "gender": "ชาย",
    "age": "24",
    "symptoms": "แสบอก, แสบคอ, ท้องอืด, อาหารไม่ย่อย, จุกคอ",
    "question": "มีคืนนึงตอนผมกำลังจะนอนจู่ๆผมก็รู้สึกแสบคอขึ้นมา แล้วพอตื่นมาก็รู้สึกว่าเหมือนมีก้อนอะไรมาจุกที่คอ ผมเลยค้นหาจากอินเตอร์เน็ตก็พบว่าผมเป็นกรดไหลย้อนจึงไปซื้อยาที่ร้านขายยามากินเป็นเวลา 1 อาทิตย์ จนพอผมเริ่มรู้สึกดีขึ้นผมจึงไปกินบุฟเฟ่ต์ชาบูแล้วทีนี้ผมรู้สึกว่าอาหารมันไม่ย่อยเลยแม้แต่ตอนกำลังจะนอนก็รู้สึกได้ว่ามีอาหารอยู่ในท้องจนถึงวัดถัดไป ควรรักษาต่อยังไงดีครับ"
})

print(my_question)

คำตอบจากแพทย์ผู้เชี่ยวชาญ

สาเหตุที่เป็นไปได้: อาการแสบอก, แสบคอ, ท้องอืด, อาหารไม่ย่อย, จุกคอที่ผู้ป่วยประสบ อาจเกิดจากกรดไหลย้อน (GERD) ซึ่งเป็นภาวะที่กรดในกระเพาะอาหารไหลย้อนกลับขึ้นไปในหลอดอาหาร ทำให้เกิดอาการแสบคอและจุกแน่นท้อง

แนวทางการดูแลเบื้องต้น: ในกรณีนี้ ผู้ป่วยควรปรึกษาแพทย์เพื่อรับการวินิจฉัยและการรักษาที่เหมาะสม โดยอาจเริ่มจากการใช้ยาลดกรด เช่น omeprazole หรือ ranitidine หากอาการไม่ดีขึ้นหรือมีอาการรุนแรงขึ้น ควรไปพบแพทย์เพื่อตรวจเพิ่มเติม

คำแนะนำเพิ่มเติม: ผู้ป่วยควรหลีกเลี่ยงอาหารที่กระตุ้นให้เกิดกรดไหลย้อน เช่น อาหารรสเผ็ดจัด, น้ำอัดลม, และกาแฟ นอกจากนี้ ควรปรับเปลี่ยนพฤติกรรมการนอน โดยไม่ควรนอนหลังทานอาหารอย่างน้อย 2-3 ชั่วโมง

คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ 
https://bit.ly/doctor-refer-agnos


In [ ]:
my_question_x = rag_chain_x.invoke({    
    "subject": "กรดไหลย้อน อาหารไม่ย่อย",
    "gender": "ชาย",
    "age": "24",
    "symptoms": "แสบอก, แสบคอ, ท้องอืด, อาหารไม่ย่อย, จุกคอ",
    "question": "มีคืนนึงตอนผมกำลังจะนอนจู่ๆผมก็รู้สึกแสบคอขึ้นมา แล้วพอตื่นมาก็รู้สึกว่าเหมือนมีก้อนอะไรมาจุกที่คอ ผมเลยค้นหาจากอินเตอร์เน็ตก็พบว่าผมเป็นกรดไหลย้อนจึงไปซื้อยาที่ร้านขายยามากินเป็นเวลา 1 อาทิตย์ จนพอผมเริ่มรู้สึกดีขึ้นผมจึงไปกินบุฟเฟ่ต์ชาบูแล้วทีนี้ผมรู้สึกว่าอาหารมันไม่ย่อยเลยแม้แต่ตอนกำลังจะนอนก็รู้สึกได้ว่ามีอาหารอยู่ในท้องจนถึงวัดถัดไป ควรรักษาต่อยังไงดีครับ"
})

print(my_question_x)

In [ ]:
my_question2 = rag_chain.invoke({    
    "question": "สวัสดีครับคือช่วงนี้ผมนอนไม่ค่อยหลับเวลานอนมันก็จะคิดฟุ้งซ่านตลอดเวลาหรือบางทีก็มีเพลงเล่นในหัวตลอดเวลาเลยทำให้นอนไม่พอ พอจะมีวิธีไหนช่วยจัดการไหมครับ"
})

print(my_question2)

In [ ]:
my_question_x2 = rag_chain_x.invoke({    
    "question": "สวัสดีครับคือช่วงนี้ผมนอนไม่ค่อยหลับเวลานอนมันก็จะคิดฟุ้งซ่านตลอดเวลาหรือบางทีก็มีเพลงเล่นในหัวตลอดเวลาเลยทำให้นอนไม่พอ พอจะมีวิธีไหนช่วยจัดการไหมครับ"
})

print(my_question_x2)

Llama.generate: 267 prefix-match hit, remaining 3475 prompt tokens to eval


KeyboardInterrupt: 